In [13]:
%load_ext autoreload
%autoreload 2
import os
if not os.path.exists('./modules') and not os.path.exists('modules.zip'):
    from google.colab import files
    uploaded = files.upload()
if not os.path.exists('./modules') and os.path.exists('modules.zip'):
    os.system('unzip modules.zip -d .')

!pip3 install optuna
!pip3 install kymatio
import torch
import torch.nn as nn
import optuna
from modules import Trainer
from modules.competition_dataset import EEGDataset, LABELS
from modules.utils import split_and_get_loaders, evaluate_model, get_closest_divisor
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import numpy as np
from sklearn.manifold import TSNE
from torchsummary import summary
from sklearn.feature_selection import f_classif
from torch.utils.data import ConcatDataset, random_split, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


device(type='cpu')

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = '/content/drive/MyDrive/ai_data/eeg_detection/data/mtcaic3'
# model_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/models/ssvep.pth'
# optuna_db_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/optuna/optuna_studies.db'
data_path = './data/mtcaic3'
model_path = './checkpoints/ssvep/models/the_great_one.pth'
pca_model_path = './checkpoints/ssvep/models/pca.pkl'
ica_model_path = './checkpoints/ssvep/models/ica.pkl'
optuna_db_path = './checkpoints/ssvep/optuna/2_optuna_studies.db'

In [6]:
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [ ]:
window_length = 128 * 2 # ensure divisble by 64 the kernel size
print(window_length)
stride = window_length // 3
batch_size = 64

dataset_train = EEGDataset(
    data_path,
    window_length=window_length,
    stride=stride,
    domain="time",
    data_fraction=1.0,
    hardcoded_mean=True,
)

dataset_val = EEGDataset(
    data_path=data_path,
    window_length=window_length,
    stride=stride,
    task='ssvep',
    split='validation',
    read_labels=True,
    hardcoded_mean=True,
)

combined = ConcatDataset([dataset_train, dataset_val])
train_len = int(len(combined) * 0.8)
val_len = len(combined) - train_len
train_ds, val_ds = random_split(combined, [train_len, val_len])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size)

256
skipped: 0/2400
data shape: (41141, 2, 256), mean shape: (1, 2, 1)
skipped: 0/50
data shape: (842, 2, 256), mean shape: (1, 2, 1)


In [17]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class DepthWiseConv2D(nn.Module):
    def __init__(self, in_channels, kernel_size, dim_mult=1, padding=0, bias=False):
        super(DepthWiseConv2D, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels * dim_mult, padding=padding, kernel_size=kernel_size, groups=in_channels, bias=bias)

    def forward(self, x: torch.Tensor):
        return self.depthwise(x)


class SeperableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, bias=False):
        super(SeperableConv2D, self).__init__()
        self.depthwise = DepthWiseConv2D(in_channels, kernel_size, padding=padding)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

class SSVEPClassifier(nn.Module):
    # EEG Net Based
    # todo look at this https://paperswithcode.com/paper/a-transformer-based-deep-neural-network-model
    def __init__(self, n_electrodes=16, out_dim=4, dropout=0.25, kernLength=256, F1=96, D=1, F2=96, hidden_dim=100, layer_dim=1):
        super().__init__()

        # B x C x T
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            #
            DepthWiseConv2D(F1, (n_electrodes, 1), dim_mult=D, bias=False),
            nn.BatchNorm2d(F1*D),
            nn.ELU(),
            nn.MaxPool2d((1, 2)), # todo try making this max pool
            nn.Dropout(dropout),
            #
            SeperableConv2D(F1 * D, F2, kernel_size=(1, 16), padding='same', bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout),
        )

        self.lstm_head = LSTMModel(F2, hidden_dim, layer_dim, out_dim)

    def forward(self, x: torch.Tensor):
        """expected input shape: BxCxT"""
        x = x.unsqueeze(1)
        y = self.block_1(x) # B x F1 x 1 x time_sub

        y = y.squeeze(2) # B x F1 x time_sub
        y = y.permute(0, 2, 1) # B x time_sub x F1
        y = self.lstm_head(y)

        return y

dummy_x = torch.randn(5, 2, 256).to(device)
model = SSVEPClassifier(
    n_electrodes=2,
    dropout=0.33066508963955576,
    kernLength=64,
    F1 = 8,
    D = 2,
    F2 = 32,
    hidden_dim=256,
    layer_dim=2,
).to(device)

model(dummy_x)

tensor([[-0.0079, -0.0211,  0.0180, -0.0745],
        [-0.0083, -0.0288,  0.0360, -0.0791],
        [-0.0072, -0.0262,  0.0315, -0.0845],
        [-0.0098, -0.0269,  0.0263, -0.0684],
        [-0.0098, -0.0373,  0.0177, -0.0746]], grad_fn=<AddmmBackward0>)

In [21]:
try:
    model.load_state_dict(torch.load(model_path, weights_only=True))
except Exception:
    print("skipping model loading...")


opt = torch.optim.Adam(model.parameters(), lr=0.00030241790493218325)
criterion = nn.CrossEntropyLoss()
avg_losses = []
val_accuracies = []

epochs = 200
for epoch in range(epochs):
    avg_loss = 0
    model.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device).to(torch.int64)
        y_pred = model(x).to(device)

        loss = criterion(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        avg_loss += loss.item()

    avg_loss /= len(train_loader)
    avg_losses.append(avg_loss)

    if epoch % 5 == 0:
        evaluation = evaluate_model(model, val_loader, device)
        val_accuracies.append(evaluation)
        model.cpu()
        torch.save(model.state_dict(), model_path)
        model.to(device)
        print(f"epoch: {epoch}, avg_loss: {avg_loss}, val_evaluation: {evaluation}")

skipping model loading...
epoch: 0, avg_loss: 1.3850079456965128, val_evaluation: 0.2889127069191378


KeyboardInterrupt: 

In [ ]:
# maxpool
plt.plot(range(len(avg_losses)), avg_losses, "b-", label="trainingg loss")
plt.plot(range(len(val_accuracies)), val_accuracies, "r-", label="validation accuracies")
plt.legend()
print(f"min avg_losses: {min(avg_losses)}")
print(f"max val_accuracies: {max(val_accuracies)}")

NameError: name 'avg_losses' is not defined

In [ ]:
class CustomTrainer(Trainer):
    def _prepare_training(self, is_trial, do_not_modify_network=True, batch_size=batch_size, window_length=window_length, stride_factor=3):
        super()._prepare_training(is_trial, do_not_modify_network, batch_size=batch_size,
                                  window_length=window_length,
                                  stride_factor=stride_factor, is_wavelet=True)
        assert self.dataset is not None

        if is_trial:
            assert isinstance(self.trial, optuna.Trial), "trial is none, cant' suggest params"

            if do_not_modify_network:
                best_params = self._get_study().best_params if do_not_modify_network else None
                assert best_params is not None, "best_params is None, can't use them"

                kernLength = best_params["kernLength"]
                F1 = best_params["F1"]
                D = best_params["D"]
                F2 = best_params["F2"]
                hidden_dim = best_params["hidden_dim"]
                layer_dim = best_params["layer_dim"]

            else:
                kernLength = self.trial.suggest_categorical("kernLength", [128, 256, 512])
                F1 = self.trial.suggest_categorical("F1", [64, 96, 128])
                D = self.trial.suggest_categorical("D", [1, 2, 3])
                F2 = self.trial.suggest_categorical("F2", [64, 96, 128])
                hidden_dim = self.trial.suggest_categorical("hidden_dim", [64, 128, 256])
                layer_dim = self.trial.suggest_categorical("layer_dim", [1, 2, 3, 4])

            dropout = self.trial.suggest_float("dropout", 0, 0.5)
            lr = self.trial.suggest_float("lr", 3e-4, 3e-2, log=True)
        else:
            # best_params = self._get_study().best_params
            # kernLength = best_params["kernLength"]
            # F1 = best_params["F1"]
            # D = best_params["D"]
            # F2 = best_params["F2"]
            # hidden_dim = best_params["hidden_dim"]
            # layer_dim = best_params["layer_dim"]
            # dropout = best_params["dropout"]
            # lr = best_params["lr"]
            dropout=0.33066508963955576
            kernLength=256
            F1 = 128
            D = 2
            F2 = F1 # OOPS 96
            hidden_dim=256
            layer_dim=3
            lr = 0.000010241790493218325

        n_samples = self.dataset.data[0].shape[1]  # data[x] shape CxT
        n_electrodes = self.dataset.data[0].shape[0]

        n_samples = self.dataset.data[0].shape[1]  # data[x] shape CxT
        n_electrodes = self.dataset.data[0].shape[0]

        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes, n_samples=n_samples, out_dim=4, dropout=dropout, kernLength=kernLength, F1=F1, D=D, F2=F2, hidden_dim=hidden_dim, layer_dim=layer_dim
        )
        print(f"lr: {lr}")
        try:
            self.model.load_state_dict(torch.load(model_path))
            print(f"loaded model weights")
        except Exception:
            print(f"no model weights found at {model_path}")
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)

trainer = CustomTrainer(data_path, optuna_db_path, model_path, train_epochs=10000, optuna_n_trials=35)

In [ ]:
delete_existing = False
trainer.optimize(delete_existing)

[I 2025-06-20 18:42:30,753] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.
[I 2025-06-20 18:42:30,815] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.
[W 2025-06-20 18:42:30,839] Trial 0 failed with parameters: {} because of the following error: ValueError('Record does not exist.').
Traceback (most recent call last):
  File "/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/home/zeyadcode/Workspace/ai_projects/eeg_detection/modules/trainer.py", line 101, in _objective
    self._prepare_training(True)
  File "/tmp/ipykernel_306525/3387801006.py", line 3, in _prepare_training
    super()._prepare_training(is_trial, do_not_modify_network, batch_size=batch_size,
  File "/home/zeyadcode/Workspace/ai_projects/eeg_detection/modules/trainer.py", li

ValueError: Record does not exist.

In [ ]:
# manual_write_study_params(trainer.study_name, trainer.storage)
trainer.train()

In [ ]:
trainer._prepare_training(False)
trainer.model.eval()
f"test accuracy: {evaluate_model(trainer.model, trainer.test_loader, device)}"

lr: 1.0241790493218325e-05
loaded model weights


'test accuracy: 0.74484375'